# Mouse Population Density Estimation

Here I'm playing around with creating a small neural network to take in the results of the simualtions and try to predict the true population density.

## Notes

I've tried using just the density estimations of each square and it didn't do much better than a linear model (somewhat unsurprisingly). I'm **now incorperating trap spacing and catch radius**. 

## To-Do

- Use some sort of inverse weighting scheme because most of the estimates are already good because the model works well. We need to emphasize the areas where the method doesn't work as well to beat a linear model.
- Try adding in other metrics (variance of the error, for instance) and maybe even make it the target for training rather than L2 error.

In [ ]:
import pandas as pd
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxboard import SummaryWriter
import numpy as np

In [ ]:
batch_size = 3500000
epochs = 100
learning_rate = 0.001

num_train_samples = 3500000
num_test_samples = 20000

context = mx.gpu()

In [ ]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(1))
    net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=context)

net

In [ ]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Lambda(lambda x: x.reshape(x.shape[0], 1, x.shape[1])))
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=3))
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=3))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(32, activation="relu"))
    net.add(gluon.nn.Dense(32, activation="relu"))
    net.add(gluon.nn.Dense(32, activation="relu"))
    net.add(gluon.nn.Dense(32, activation="relu"))
    net.add(gluon.nn.Dense(1))
    net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=context)

net

In [ ]:
net.summary(nd.random.uniform(shape=(batch_size, 8), ctx=context))

In [ ]:
simdata = pd.read_csv("data/trainingdata-random.csv")
simdata.shape

In [ ]:
simtrain = simdata.sample(n=num_train_samples)
simtest = simdata.drop(simtrain.index).sample(num_test_samples)

# predictors = ["square1", "square2","square3", "square4","square5", "square6","square7", "square8", "TrapSpacing", "CatchRadius"]
predictors = ["square1", "square2","square3", "square4","square5", "square6","square7", "square8"]

Xtrain = nd.array(simtrain[predictors], ctx=context)
Ytrain = nd.array(simtrain["Density"], ctx=context)

Xtest = nd.array(simtest[predictors], ctx=context)
Ytest = nd.array(simtest["Density"], ctx=context)

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtrain, Ytrain), batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtest, Ytest), batch_size=batch_size, shuffle=False)

In [ ]:
Xtrain.shape

In [ ]:
num_training_batches = np.ceil(Xtrain.shape[0]/batch_size)
num_training_batches

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate})

metric = mx.metric.MSE() # train metric
loss = gluon.loss.L2Loss() # L2 loss

In [ ]:
# define a summary writer that logs data and flushes to the file every 5 seconds
sw = SummaryWriter(logdir='./logs2', flush_secs=5)

In [ ]:
def test(ctx):
    metric = mx.metric.MSE()
    for data, label in test_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])

    return metric.get()

In [ ]:
# Train the network
global_step = 2000
for epoch in range(epochs):
    epoch += 2000
    # reset data iterator and metric at begining of epoch.
    metric.reset()
    
    # training loop for epoch
    for i, (data, label) in enumerate(train_data):
        # Copy data to context (ctx) if necessary
        data = data.as_in_context(context)
        label = label.as_in_context(context)
        # Start recording computation graph with record() section.
        # Recorded graphs can then be differentiated with backward.
        with autograd.record():
            output = net(data)
            L = loss(output, label)
        sw.add_scalar(tag='train_loss', value=L.mean().asscalar(), global_step=global_step)
        global_step += 1
        L.backward()

        # take a gradient step with batch_size equal to data.shape[0]
        trainer.step(data.shape[0])
        
        # update metric at last.
        metric.update([label], [output])
        
    # logging training accuracy
    name, train_acc = metric.get()
    sw.add_scalar(tag='accuracy_curves', value=('train_acc', train_acc), global_step=epoch)
    
    # logging testing accuracy
    name, test_acc = test(context)
    sw.add_scalar(tag='accuracy_curves', value=('valid_acc', test_acc), global_step=epoch)
        
    # Record input and output samples
    # sample 10 random inputs and outputs
    sampleidx = nd.random_randint(low=0, high=Ytest.shape[0], shape=10, ctx=context)
    Xsamples = Xtest[sampleidx]
    output_samples = np.array2string(net(Xsamples)[:,0].asnumpy())
    expected_ouput = np.array2string(Ytest[sampleidx].asnumpy())
#     sw.add_text(tag='output_samples', text=output_samples, global_step=epoch)
#     sw.add_text(tag='expected_ouput', text=expected_ouput, global_step=epoch)
    if epoch % 10 == 0:
        print("Output:", output_samples)
        print("Expected:", expected_ouput)
    
sw.export_scalars('scalar_dict.json')
sw.close()

In [ ]:
sw.close()

In [ ]:
net.save_parameters("8squaresTSCR-firstmodel-datarand.param")

In [ ]:
sampleidx = nd.random_randint(low=0, high=Ytest.shape[0], shape=10, ctx=context)
sampleidx

In [ ]:
Xtest[sampleidx]

In [ ]:
net(Xtest[sampleidx])

In [ ]:
test(context)

# Alternate Approach

Here I'm fitting the same data but using a random forest rather than a neural network.

In [ ]:
import pandas as pd
simdata = pd.read_csv("data/trainingdata-random.csv")
simdata.shape

simtrain = simdata.sample(n=100000)
simtest = simdata.drop(simtrain.index).sample(10000)

In [ ]:
Xtrain = simtrain[simdata.columns.difference(["Density", "uuid"])]
ytrain = simtrain["Density"]

Xtest = simtest[simdata.columns.difference(["Density", "uuid"])]
ytest = simtest["Density"]

In [ ]:
Xtrainrf = Xtrain.asnumpy()
ytrainrf = Ytrain.asnumpy()
Xtestrf = Xtest.asnumpy()
ytestrf = Ytest.asnumpy()

In [ ]:
print(Xtrain.shape)
print(Ytrain.shape)
print(Xtest.shape)
print(Ytest.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=123, n_jobs=-1)
reg = rf.fit(Xtrainrf, ytrainrf)

In [ ]:
train_mse = mean_squared_error(ytrainrf, reg.predict(Xtrainrf))
test_mse = mean_squared_error(ytestrf, reg.predict(Xtestrf))
print("train mse:", train_mse, "test mse:", test_mse)

## Results

| Method | Param               | MSE obtained |
|--------|---------------------|--------------|
| RF     | 100 trees, 20 depth | 0.1297       |
| RF     | 100 trees, 10 depth | 0.1516       |
| NN     |                     |              |